In [2]:
import torch

from pathlib import Path

from app.model.config import Config
from app.model.time_series_embedding import TimeSeriesEmbedding

data_dir = Path('app/data')
config = Config(data_dir)
time_series_embedding = TimeSeriesEmbedding(config)
input = torch.randn(32, config.seq_len, config.input_dims)
output = time_series_embedding(input)

display(f'{input.shape=}; {output.shape=}')

'input.shape=torch.Size([32, 5, 5]); output.shape=torch.Size([32, 5, 150])'

In [3]:
import torch
from app.model.config import Config
from app.model.positional_encoding import PositionalEncoding

data_dir = Path('app/data')
config = Config(data_dir)
positional_encoding = PositionalEncoding(config)
input = torch.randn((32, config.seq_len, config.d_model))
output = positional_encoding(input)

display(f'{input.shape=}; {output.shape=}')

'input.shape=torch.Size([32, 5, 150]); output.shape=torch.Size([32, 5, 150])'

In [4]:
import torch
from app.model.config import Config
from app.model.stock_transformer import StockTransformer


data_dir = Path('app/data')
config = Config(data_dir)
input = torch.randn((config.batch_size, config.seq_len, config.input_dims),
                    dtype=torch.float)
display(f'{input.shape=}')
transformer = StockTransformer(config)
output = transformer(input)
display(f'{output.shape=}')

'input.shape=torch.Size([16, 5, 5])'

'output.shape=torch.Size([16, 1])'

In [5]:
import csv
import yfinance as yf

from pathlib import Path


ticker = 'SPY'
data = yf.Ticker(ticker)
hist = data.history(period='36mo', interval='1d')

if hist.empty:
    print(f'No data available for {ticker}')
else:
    date_from = hist.index.min().strftime('%Y%m%d')
    date_to = hist.index.max().strftime('%Y%m%d')

    out_dir = Path('app/data') / 'tickers'
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / f"{ticker}_{date_from}_{date_to}.csv"

    columns = ['Open', 'High', 'Low', 'Volume', 'Close']
    with out_path.open('w', newline='') as fp:
        writer = csv.writer(fp)
        writer.writerow(columns)
        for row in hist.itertuples():
            writer.writerow([getattr(row, c) for c in columns])

    print(f'Saved {out_path}')

Saved app/data/tickers/SPY_20230103_20251231.csv


In [8]:
from pathlib import Path

from app.model.checkpointer import Checkpointer


model_path = Path('app/data/models/1000/SPY.pkl')
model, config, step = Checkpointer.load_checkpoint(model_path)
display(model, config, step)

Model loaded from app/data/models/1000/SPY.pkl


StockTransformer(
  (time_series_embedding): TimeSeriesEmbedding(
    (feature_projection): Linear(in_features=5, out_features=150, bias=True)
  )
  (positional_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-4): 5 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=150, out_features=150, bias=True)
        )
        (linear1): Linear(in_features=150, out_features=300, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=300, out_features=150, bias=True)
        (norm1): LayerNorm((150,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((150,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (regressor): Linear(in_features=150, out

1000